In [36]:
import numpy as np
import pandas as pd

from arch import arch_model



In [88]:
# CREATES DATAFRAME FROM BACEN-SGS SERIES
def data_frame(names = list(),
               numbers = list(),
               initial_date = str(),
               final_date = str()):
    for i in range(len(names)):
        name = str(names[i])
        url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=csv&&dataInicial={}&dataFinal={}'.format(numbers[i], initial_date, final_date)
        df = pd.read_csv(url, sep = ';', index_col = 0, parse_dates = [0], infer_datetime_format = True, decimal = ',')
        if i == 0:
            DF = pd.DataFrame({name: df.valor},
                              index = df.index)
        else:
            DF[name] = df.valor
    return(DF)

# APPENDS EXCHANGE COUPON TO DATAFRAME
def exchange_coupon(df = pd.DataFrame(),
                  dol = int(),
                  rs = list(),
                  names = list()):
    usd = df[df.columns[dol]]
    for e in range(len(rs)):
        r = df[df.columns[rs[e]]]
        name = names[e]
        arr = np.array(list())
        for i in range(len(usd)):
            if i == 0:
                arr = np.append(arr, np.NaN)
            else:
                arr = np.append(arr, (r[i]/100)/(usd[i]/usd[i-1]))
        df[name] = arr

# APPENDS GARCH CSD AND RESIDUALS TO DATAFRAME
def garch(df = pd.DataFrame(),
          cols = list()):
    for i in range(len(cols)):
        name = df.columns[cols[i]]
        fitted_model = arch_model(df[name][1:]).fit()
        df['{}Csd'.format(name)] = fitted_model.conditional_volatility
        df['{}Res'.format(name)] = fitted_model.resid

# APPENDS PARAMETRIC AND NON PARAMETRIC LIMITS TO DATAFRAME
from scipy import stats
def limits(df = pd.DataFrame(),
           cols = list()):
    for e in range(len(cols)):
        name = df.columns[cols[e]]
        series = df[name]
        # PARAMETRIC
        mean = series.mean()
        std = series.std()
        # ----UPPER
        upper = mean + stats.norm.ppf(q = 0.975) * (std)
        arr = np.array(list())
        for i in range(len(series)):
            if i == 0:
                arr = np.append(arr, np.NaN)
            else:
                arr = np.append(arr, upper)
        df['{}ParUp'.format(name)] = arr
        # ----LOWER 
        lower = mean - stats.norm.ppf(q = 0.975) * (std)
        arr = np.array(list())
        for i in range(len(series)):
            if i == 0:
                arr = np.append(arr, np.NaN)
            else:
                arr = np.append(arr, lower)
        df['{}ParLo'.format(name)] = arr
        # NON PARAMETRIC
        mean = series.rolling(window = 63, min_periods = 0, center = True).mean()
        std = series.rolling(window = 63, min_periods = 0, center = True).std()
        # ----UPPER
        arr = np.array(list())
        for i in range(len(mean)):
            upper = mean[i] + stats.norm.ppf(q = 0.975) * (std[i])
            if i == 0:
                arr = np.append(arr, np.NaN)
            else:
                arr = np.append(arr, upper)
        df['{}NonUp'.format(name)] = arr
        # ----LOWER
        arr = np.array(list())
        for i in range(len(mean)):
            lower = mean[i] - stats.norm.ppf(q = 0.975) * (std[i])
            if i == 0:
                arr = np.append(arr, np.NaN)
            else:
                arr = np.append(arr, lower)
        df['{}NonLo'.format(name)] = arr

In [89]:
BASE = data_frame(names = ['Ptax', 'Selic', 'Di'],
                  numbers = [1, 11, 12],
                  initial_date = '26/09/2016',
                  final_date = '16/05/2019')

exchange_coupon(BASE, 0, [1, 2], ['Oc1', 'Di1'])

garch(BASE, [3,4])

limits(BASE, [5, 7])

BASE.head()

Iteration:      1,   Func. Count:      6,   Neg. LLF: -5286.810268575844
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5286.810271489501
            Iterations: 1
            Function evaluations: 6
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      6,   Neg. LLF: -5287.985826710033
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -5287.985829620425
            Iterations: 1
            Function evaluations: 6
            Gradient evaluations: 1


/Users/bernardopaulsen/anaconda3/lib/python3.6/site-packages/arch/univariate/base.py:571: ConvergenceWarning: 
The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)
/Users/bernardopaulsen/anaconda3/lib/python3.6/site-packages/arch/univariate/base.py:571: ConvergenceWarning: 
The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning)


,Ptax,Selic,Di,Oc1,Di1,Oc1Csd,Oc1Res,Di1Csd,Di1Res,Oc1CsdParUp,Oc1CsdParLo,Oc1CsdNonUp,Oc1CsdNonLo,Di1CsdParUp,Di1CsdParLo,Di1CsdNonUp,Di1CsdNonLo
data,,,,,,,,,,,,,,,,,
2016-09-26,3.2400,0.052531,0.052461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-09-27,3.2358,0.052531,0.052461,0.000526,0.000525,0.000195,0.000201,0.000195,0.000200,0.000174,0.000009,0.000199,0.000182,0.000174,0.000009,0.000199,0.000182
2016-09-28,3.2476,0.052531,0.052461,0.000523,0.000523,0.000195,0.000198,0.000195,0.000198,0.000174,0.000009,0.000200,0.000181,0.000174,0.000009,0.000199,0.000181
2016-09-29,3.2235,0.052531,0.052461,0.000529,0.000529,0.000194,0.000204,0.000194,0.000204,0.000174,0.000009,0.000200,0.000180,0.000174,0.000009,0.000199,0.000180
2016-09-30,3.2462,0.052531,0.052461,0.000522,0.000521,0.000195,0.000196,0.000194,0.000196,0.000174,0.000009,0.000200,0.000180,0.000174,0.000009,0.000199,0.000179
